In [4]:
import re

exps = []
with open('input', 'r') as f:
    for line in f:
        exps.append(line.strip())

ops = {
    '+': (lambda x,y: x + y),
    '*': (lambda x,y: x * y)
}

def findClosingBracket(exp):
    lvl = 1
    i = 1
    for c in exp[1:]:
        if c == '(':
            lvl += 1
        elif c == ')':
            lvl -= 1
        
        if lvl == 0:
            return i
        i += 1
    return None

In [5]:
from itertools import takewhile
verbose = False

def num(exp):
    exp = exp.strip()
    if exp and exp[0].isdigit():
        num = ''.join(list(takewhile(lambda c: c.isdigit(), exp)))
        return (int(num), exp[len(num):])
    else:
        None
        
def op(exp):
    exp = exp.strip()
    if exp and exp[0] in ['*', '+']:
        return (exp[0],exp[1:].strip())
    else:
        return None

def braced(exp):
    exp = exp.strip()
    if exp and exp[0] == '(':
        i = findClosingBracket(exp)
        return (exp[1:i], exp[i+1:])
    else:
        return None

def term(exp):
    exp = exp.strip()
    if exp:
        return num(exp) or braced(exp) or None
    else:
        return None
    
def nexp(exp,n=None):
    left, operand, right, rest = None, None, None, None
    if not n:
        left, rest = term(exp) or (None,None)
        verbose and print(f'left={left} rest={rest}')
    else:
        left, rest = n, exp
    if rest:
        operand,rest = op(rest) or (None,None)
        verbose and print(f'op={operand} rest={rest}')
    if operand:
        right, rest = term(rest) or (None,None)
        verbose and print(f'right={right} rest={rest}')
    if left:
        if op and right:
            l = evaluate(left)
            r = evaluate(right)
            verbose and print(f'{l} {operand} {r} => {ops[operand](l,r)}')
            return nexp(rest, ops[operand](l,r))
        else:
            return left
    return n
def evaluate(exp):
    if type(exp) == int:
        return exp
    else:
        return nexp(exp)

sols=[]
for exp in exps:
    print(f'"{exp}"')
    sol = nexp(exp)
    print(f'{sol}')
    sols.append(sol)
    print('-'*30)
sum(sols)

"4 * ((9 + 9 * 5 + 5) + 3)"
392
------------------------------
"4 + 5 * (3 + 4 * 3 * 8 * 4 + (5 * 4 + 8)) * 6"
37800
------------------------------
"3 * (6 + (6 * 9 * 9) + (3 + 7 * 9 + 8 * 9) * 3 * 7) + 5 + 2 * (3 + 9 * 9)"
9349452
------------------------------
"((2 + 4) + 7) * (8 * 6 + 9 * (9 + 7 * 3 + 9 + 6) + (8 * 9 + 2 + 8)) * 2 + 9"
95507
------------------------------
"2 + ((8 + 3 + 6 + 6 * 7 * 8) * 3 * (6 + 4 * 3 + 5 * 8 * 8)) + 2 + (8 * 2 * 3) + (3 * 4 * 5 + 4) + 5"
8655481
------------------------------
"8 * 7 + (3 + 4) + (9 + 3 + 3 + 3 * 6 * 9)"
1035
------------------------------
"(2 + 3 + 9 + 7 * (2 + 9 + 3 * 9) * (3 * 8 + 9 + 3 + 9 + 5)) * 8 + (2 + 5 + 5) + 9 * 3"
3175263
------------------------------
"6 * 3 * 7 * 7 + (5 * 5)"
907
------------------------------
"2 * (7 * 3 * 6 + 7 * (4 * 2 + 7)) * 5 * 3 + 2"
59852
------------------------------
"6 * 7 + 8 + (6 + (4 * 9 + 9 * 2 * 7) + (9 + 7 + 2))"
704
------------------------------
"6 * 5 + 9 * 7 + 7 * (8 + 6 * 8 + (3 + 

11297104473091

In [133]:
import parsley
from functools import reduce
import operator as op

class Exp:
    def __init__(self):
        pass
    def evaluate(self):
        pass
    def unpack_if(self,typedef):
        return self

class Num(Exp):
    def __init__(self,n):
        self.n = n

    def evaluate(self):
        return self.n
    def __repr__(self):
        return f'N({self.n})'
    
class OP(Exp):
    def __init__(self,symbol,operands):
        self.symbol = symbol
        self.operands = operands
        self.op = (lambda operands: None)
    
    def unpack_if(self,typedef):
        return self.operands if all([typedef == type(o) for o in self.operands + [self]]) else self
    
    def evaluate(self):
        return self.op([o.evaluate() for o in self.operands])
    

class Mul(OP):
    def __init__(self, factors):
        super().__init__('*',factors)
        self.op = (lambda f: reduce(op.mul, f))
        
    def __repr__(self):
        return f'Mul({self.operands})'
        
class Plus(OP):
    def __init__(self, sums):
        super().__init__('+',sums)
        self.op = (lambda s: sum(s))
        
    def __repr__(self):
        return f'Plus({self.operands})'

class Brace(Exp):
    def __init__(self,operation):
        self.operation = operation
    
    def __repr__(self):
        return f'B<{self.operation}>'
    
    def evaluate(self):
        return self.operation.evaluate()

g = parsley.makeGrammar(
# standard arithmetic
#exp = ( exp:left ('+' value)+:right -> Plus([left] + right)
#      | exp:left ('*' value)+:right -> Mul([left] + right)
#      | value:v             -> v)
"""
number  = ws <digit+>:d ws -> Num(int(d))
bexp    = ws '(' ws exp:expression ws ')' ws -> Brace(expression)
value = number | bexp
ws = ' '*
exp = ( exp:left '+' (value)+:right -> Plus([left] + right)
      | exp:left '*' exp:right -> Mul([left, right])
      | value:v             -> v)
""",{'Plus': Plus, 'Mul': Mul, 'Brace': Brace, 'Num': Num})
sols=[]
for exp in exps:
    print(f'"{exp}"')
    sol = g(exp).exp()
    print(f'{sol}')
    sols.append(sol.evaluate())
    print('-'*30)
sum(sols)

"4 * ((9 + 9 * 5 + 5) + 3)"
Mul([N(4), B<Plus([B<Mul([Plus([N(9), N(9)]), Plus([N(5), N(5)])])>, N(3)])>])
------------------------------
"4 + 5 * (3 + 4 * 3 * 8 * 4 + (5 * 4 + 8)) * 6"
Mul([Plus([N(4), N(5)]), Mul([B<Mul([Plus([N(3), N(4)]), Mul([N(3), Mul([N(8), Plus([N(4), B<Mul([N(5), Plus([N(4), N(8)])])>])])])])>, N(6)])])
------------------------------
"3 * (6 + (6 * 9 * 9) + (3 + 7 * 9 + 8 * 9) * 3 * 7) + 5 + 2 * (3 + 9 * 9)"
Mul([N(3), Mul([Plus([Plus([B<Mul([Plus([Plus([N(6), B<Mul([N(6), Mul([N(9), N(9)])])>]), B<Mul([Plus([N(3), N(7)]), Mul([Plus([N(9), N(8)]), N(9)])])>]), Mul([N(3), N(7)])])>, N(5)]), N(2)]), B<Mul([Plus([N(3), N(9)]), N(9)])>])])
------------------------------
"((2 + 4) + 7) * (8 * 6 + 9 * (9 + 7 * 3 + 9 + 6) + (8 * 9 + 2 + 8)) * 2 + 9"
Mul([B<Plus([B<Plus([N(2), N(4)])>, N(7)])>, Mul([B<Mul([N(8), Mul([Plus([N(6), N(9)]), Plus([B<Mul([Plus([N(9), N(7)]), Plus([Plus([N(3), N(9)]), N(6)])])>, B<Mul([N(8), Plus([Plus([N(9), N(2)]), N(8)])])>])])])>, Plus([

"((8 * 5 + 4 * 4) + (6 + 5 * 3 * 5 * 3) * (2 * 7) + (9 + 8 + 3 + 9 + 9 + 6)) * (6 + 9 * 3 + 4 * (7 * 2)) * 5 * 6"
Mul([B<Mul([Plus([B<Mul([N(8), Mul([Plus([N(5), N(4)]), N(4)])])>, B<Mul([Plus([N(6), N(5)]), Mul([N(3), Mul([N(5), N(3)])])])>]), Plus([B<Mul([N(2), N(7)])>, B<Plus([Plus([Plus([Plus([Plus([N(9), N(8)]), N(3)]), N(9)]), N(9)]), N(6)])>])])>, Mul([B<Mul([Plus([N(6), N(9)]), Mul([Plus([N(3), N(4)]), B<Mul([N(7), N(2)])>])])>, Mul([N(5), N(6)])])])
------------------------------
"(5 + 4 + 4 * (9 * 2 + 8 + 7 * 9) * (2 * 3 + 3 * 6 + 8) * 7) * 9 + 5"
Mul([B<Mul([Plus([Plus([N(5), N(4)]), N(4)]), Mul([B<Mul([N(9), Mul([Plus([Plus([N(2), N(8)]), N(7)]), N(9)])])>, Mul([B<Mul([N(2), Mul([Plus([N(3), N(3)]), Plus([N(6), N(8)])])])>, N(7)])])])>, Plus([N(9), N(5)])])
------------------------------
"9 + 4 + 7 + 6 * 9 + 9"
Mul([Plus([Plus([Plus([N(9), N(4)]), N(7)]), N(6)]), Plus([N(9), N(9)])])
------------------------------
"7 + 7 * 5 * (7 * 2 * 2 * 8 + 5) + 6 * 7"
Mul([Plus([N(7), N

Mul([Plus([B<Mul([Plus([B<Mul([N(2), N(7)])>, N(8)]), B<Mul([N(2), Plus([Plus([N(4), N(2)]), N(9)])])>])>, B<Mul([N(9), Plus([B<Mul([N(9), Mul([N(2), N(5)])])>, N(8)])])>]), Mul([Plus([B<Plus([B<Mul([Plus([Plus([N(7), N(2)]), N(3)]), N(3)])>, N(4)])>, B<Mul([Plus([Plus([Plus([N(5), N(4)]), N(5)]), N(8)]), Plus([N(8), N(7)])])>]), Plus([N(9), N(8)])])])
------------------------------
"((9 + 4 + 2 + 9) + 3 + 8 + (6 * 2 + 2 + 6 * 2 * 6) + 6) * 2"
Mul([B<Plus([Plus([Plus([Plus([B<Plus([Plus([Plus([N(9), N(4)]), N(2)]), N(9)])>, N(3)]), N(8)]), B<Mul([N(6), Mul([Plus([Plus([N(2), N(2)]), N(6)]), Mul([N(2), N(6)])])])>]), N(6)])>, N(2)])
------------------------------
"6 + 4 + 3 * 6 + 8"
Mul([Plus([Plus([N(6), N(4)]), N(3)]), Plus([N(6), N(8)])])
------------------------------
"4 * 7 + 5"
Mul([N(4), Plus([N(7), N(5)])])
------------------------------
"(3 + (7 + 8 + 7 * 7) * 4) + (9 * 6 * 6 + 9) + ((3 * 3 * 2 * 9 * 3 + 7) + 3 * (5 + 4 + 9 * 8 * 7) + (4 + 7 + 9 + 9) * 8 + 3)"
Plus([Plus([B<Mul

Plus([Plus([Plus([Plus([N(4), N(5)]), N(2)]), N(6)]), B<Mul([Plus([Plus([N(4), N(7)]), N(3)]), Plus([N(6), N(2)])])>])
------------------------------
"(9 + 4 + 8) + (2 * 3 + 5 + 3) + 3"
Plus([Plus([B<Plus([Plus([N(9), N(4)]), N(8)])>, B<Mul([N(2), Plus([Plus([N(3), N(5)]), N(3)])])>]), N(3)])
------------------------------
"6 * ((6 + 3 + 6 * 6 * 2 + 3) + (5 + 4 * 9 * 8 * 4 + 2) * (2 * 4 * 5) + 3) + 9 * 2 + ((6 * 9 + 6) + 3) + 5"
Mul([N(6), Mul([Plus([B<Mul([Plus([B<Mul([Plus([Plus([N(6), N(3)]), N(6)]), Mul([N(6), Plus([N(2), N(3)])])])>, B<Mul([Plus([N(5), N(4)]), Mul([N(9), Mul([N(8), Plus([N(4), N(2)])])])])>]), Plus([B<Mul([N(2), Mul([N(4), N(5)])])>, N(3)])])>, N(9)]), Plus([Plus([N(2), B<Plus([B<Mul([N(6), Plus([N(9), N(6)])])>, N(3)])>]), N(5)])])])
------------------------------
"(6 + (7 * 8) + 8) * 2 + 2 + 5 + (3 + 2 * 8)"
Mul([B<Plus([Plus([N(6), B<Mul([N(7), N(8)])>]), N(8)])>, Plus([Plus([Plus([N(2), N(2)]), N(5)]), B<Mul([Plus([N(3), N(2)]), N(8)])>])])
-------------------

Mul([N(3), Mul([N(8), Plus([Plus([N(9), N(3)]), N(2)])])])
------------------------------
"(7 + (9 + 7 * 8 * 4) * 2 * 6 + 8 + 5) + 3"
Plus([B<Mul([Plus([N(7), B<Mul([Plus([N(9), N(7)]), Mul([N(8), N(4)])])>]), Mul([N(2), Plus([Plus([N(6), N(8)]), N(5)])])])>, N(3)])
------------------------------
"8 + 9 * (2 * 8 + (7 * 9 + 8 + 4 * 2) * (5 * 6) * (4 + 4 + 6 + 4 * 3 + 8))"
Mul([Plus([N(8), N(9)]), B<Mul([N(2), Mul([Plus([N(8), B<Mul([N(7), Mul([Plus([Plus([N(9), N(8)]), N(4)]), N(2)])])>]), Mul([B<Mul([N(5), N(6)])>, B<Mul([Plus([Plus([Plus([N(4), N(4)]), N(6)]), N(4)]), Plus([N(3), N(8)])])>])])])>])
------------------------------
"(2 + 8 + 3 + (6 + 7 * 4) * 5 * 9) * (4 * 8 * 9)"
Mul([B<Mul([Plus([Plus([Plus([N(2), N(8)]), N(3)]), B<Mul([Plus([N(6), N(7)]), N(4)])>]), Mul([N(5), N(9)])])>, B<Mul([N(4), Mul([N(8), N(9)])])>])
------------------------------
"3 * 3 * 6 + 3 * (8 + 9 * 9 * 5 * 5 + 2) + 6"
Mul([N(3), Mul([N(3), Mul([Plus([N(6), N(3)]), Plus([B<Mul([Plus([N(8), N(9)]), Mul([N(

Mul([B<Mul([Plus([N(4), N(9)]), Plus([Plus([B<Mul([N(3), Mul([N(7), Mul([N(2), Mul([N(6), Plus([N(2), N(4)])])])])])>, B<Mul([N(9), Plus([N(7), N(6)])])>]), N(4)])])>, Plus([N(4), B<Mul([Plus([N(6), B<Mul([N(5), Mul([N(2), Mul([N(5), Mul([N(7), Mul([N(8), N(2)])])])])])>]), Plus([N(9), N(7)])])>])])
------------------------------
"(2 * 4) * 8"
Mul([B<Mul([N(2), N(4)])>, N(8)])
------------------------------
"6 * (2 * 8 * 5 + (7 * 3 + 6) * 8) + 7 * 5"
Mul([N(6), Mul([Plus([B<Mul([N(2), Mul([N(8), Mul([Plus([N(5), B<Mul([N(7), Plus([N(3), N(6)])])>]), N(8)])])])>, N(7)]), N(5)])])
------------------------------
"(6 + 6 + 3 + 4 + 3 + 3) + 7 + 3 * 6"
Mul([Plus([Plus([B<Plus([Plus([Plus([Plus([Plus([N(6), N(6)]), N(3)]), N(4)]), N(3)]), N(3)])>, N(7)]), N(3)]), N(6)])
------------------------------
"(5 * 8 * 3 + (6 * 5 + 5 + 2 + 5) + 2) * 4 + 4 * (4 * (6 + 6 * 8)) + 5"
Mul([B<Mul([N(5), Mul([N(8), Plus([Plus([N(3), B<Mul([N(6), Plus([Plus([Plus([N(5), N(5)]), N(2)]), N(5)])])>]), N(2)])])])

Mul([Plus([N(7), N(6)]), Mul([N(6), Plus([N(9), B<Mul([Plus([Plus([Plus([N(7), N(7)]), N(4)]), N(8)]), B<Plus([N(3), N(9)])>])>])])])
------------------------------
"4 * ((2 + 7 * 5 + 6) + 8 + 5 + 7 * 7) * 8"
Mul([N(4), Mul([B<Mul([Plus([Plus([Plus([B<Mul([Plus([N(2), N(7)]), Plus([N(5), N(6)])])>, N(8)]), N(5)]), N(7)]), N(7)])>, N(8)])])
------------------------------
"(6 * 9 * (7 * 3 + 8 + 3 * 3) + 4 * 6 * 4) * 3 * 6 + 9 + 3 + 6"
Mul([B<Mul([N(6), Mul([N(9), Mul([Plus([B<Mul([N(7), Mul([Plus([Plus([N(3), N(8)]), N(3)]), N(3)])])>, N(4)]), Mul([N(6), N(4)])])])])>, Mul([N(3), Plus([Plus([Plus([N(6), N(9)]), N(3)]), N(6)])])])
------------------------------
"5 * 4 + 7 * (2 * 3) + (8 * 2 + 5) * 7"
Mul([N(5), Mul([Plus([N(4), N(7)]), Mul([Plus([B<Mul([N(2), N(3)])>, B<Mul([N(8), Plus([N(2), N(5)])])>]), N(7)])])])
------------------------------
"2 * 7 * 8 * (4 + 3 + 4 + 6)"
Mul([N(2), Mul([N(7), Mul([N(8), B<Plus([Plus([Plus([N(4), N(3)]), N(4)]), N(6)])>])])])
-------------------------

Mul([N(4), Mul([B<Mul([B<Mul([Plus([N(5), N(9)]), Mul([N(5), Mul([N(6), N(9)])])])>, N(3)])>, Mul([N(8), B<Plus([N(4), B<Mul([N(7), Mul([N(7), Mul([N(9), N(6)])])])>])>])])])
------------------------------
"9 + 9 * 9 * (4 * 6 * 4 * 7 + 7) + ((5 * 9 + 4 * 4 + 2 + 4) * 7 * 6 * 5) + 8"
Mul([Plus([N(9), N(9)]), Mul([N(9), Plus([Plus([B<Mul([N(4), Mul([N(6), Mul([N(4), Plus([N(7), N(7)])])])])>, B<Mul([B<Mul([N(5), Mul([Plus([N(9), N(4)]), Plus([Plus([N(4), N(2)]), N(4)])])])>, Mul([N(7), Mul([N(6), N(5)])])])>]), N(8)])])])
------------------------------
"4 + 4 * (4 * 5 * (6 + 9) * 5 * 5 + 2) + (4 * (8 * 2 * 9 + 8) + 2 * 9) * 3 + 2"
Mul([Plus([N(4), N(4)]), Mul([Plus([B<Mul([N(4), Mul([N(5), Mul([B<Plus([N(6), N(9)])>, Mul([N(5), Plus([N(5), N(2)])])])])])>, B<Mul([N(4), Mul([Plus([B<Mul([N(8), Mul([N(2), Plus([N(9), N(8)])])])>, N(2)]), N(9)])])>]), Plus([N(3), N(2)])])])
------------------------------
"3 * 7 + 3 * ((3 + 2 * 9 * 5 + 5) + (3 + 5 * 7 + 2 + 6) * (5 * 8 * 8 + 7 + 8 + 4)) * 4 

185348874183674